In [1]:
import json
import numpy as np
import pandas as pd 
import requests

from backoff import on_exception, expo
from ratelimit import limits, RateLimitException

In [2]:
secret = json.load(open('secret.json'))
traits_json = json.load(open('trait.json'))['data']
augments_json = json.load(open('augment.json'))['data']
hero_augments_json = json.load(open('hero augment.json'))['data']
combined_augments_json = {**augments_json, **hero_augments_json}
champion_json = json.load(open('tft-champion.json'))['data']

In [3]:
riot_api_key = secret['api_key']
player_name = secret['player_name']
player_info = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{player_name}?api_key={riot_api_key}").json()
id = player_info['id']
account_id = player_info['accountId']
puuid = player_info['puuid']

In [4]:
# return match id
match_history_1 = requests.get(f"https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start={400}&count={200}&api_key={riot_api_key}").json()
match_history_2 = requests.get(f"https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start={600}&count={200}&api_key={riot_api_key}").json()
match_history = match_history_1 + match_history_2


In [5]:
def clean_version(version_name):
    cleaned_ver_name = version_name.split('Releases/')[1].split('>')[0]
    return cleaned_ver_name

def clean_augment(player_record):
    augments = []
    augment = player_record['augments']
    for aug in augment:
        if aug == 'TFT6_Augment_BlueBattery1':
            augments.append('Blue Battery 1')
        else:
            cleaned_aug_name = combined_augments_json[aug]['name']
            augments.append(cleaned_aug_name)
    return augments

def get_trait_list(traits_json):
    trait_list = []
    for trait_key in list(traits_json.keys()):
        trait_list.append('Trait_' + traits_json[trait_key]['name'])
    return trait_list 

def get_champion_list(champion_json):
    champion_list = []
    for champion_key in list(champion_json.keys()):
        champion_list.append('Unit_' + champion_json[champion_key]['name'])
    return champion_list 

def trait_info(player_record):
    cleaned_trait_names = []
    for i in np.arange(0, len(player_record['traits'])):
        trait = player_record['traits'][i]
        if trait['tier_current'] == 0:
            continue
        cleaned_trait_names.append(traits_json[trait['name']]['name'] + ' ' +str(trait['tier_current']))
    return cleaned_trait_names

def unit_info(player_record):
    units_name = []
    for i in np.arange(0, len(player_record['units'])):
        unit = player_record['units'][i]
        units_name.append(unit['character_id'].split('TFT8_')[1] + ' ' + str(unit['tier']))
    return units_name

trait_list = ['trait_' + str(i) for i in np.arange(1,15)]
champion_list = ['unit_' + str(i) for i in np.arange(1,20)]

df = pd.DataFrame(columns=['version_name', 'augment_1','augment_2', 'augment_3', 'placement','top 4', 'level', 'last_round', 'time_eliminated'] + trait_list + champion_list)


In [6]:
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=49, period=60)
def call_riot_api():
    response = requests.get(f"https://americas.api.riotgames.com/tft/match/v1/matches/{match_id}?api_key={riot_api_key}")

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))
    return response

In [7]:
for match_id in match_history:
    match_info = call_riot_api().json()
    if match_info['info']['tft_set_core_name'] != 'TFTSet8':
        continue
    if match_info['info']['tft_game_type'] != 'standard':
        continue
    for i in np.arange(0,8):
        participant_puuid = match_info['info']['participants'][i]['puuid']
        if participant_puuid == puuid:
            player_record = match_info['info']['participants'][i]
            break

    entries = {
           'version_name': match_info['info']['game_version'].split('<Releases/')[1].split('>')[0],
           'augment_1': clean_augment(player_record)[0],
           'augment_2': clean_augment(player_record)[1],
           'augment_3': clean_augment(player_record)[2],
           'placement': player_record['placement'],
           'top 4': True if player_record['placement'] <= 4 else False,
           'level': player_record['level'],
           'last_round': player_record['last_round'],
           'time_eliminated': round(player_record['time_eliminated'])
           }
    traits = trait_info(player_record)
    units = unit_info(player_record)


    count = 1
    for single_trait in traits: 
        entries['trait_' + str(count)] = single_trait
        count = count + 1

    count = 1
    for single_unit in units: 
        entries['unit_' + str(count)] = single_unit
        count = count + 1

    df = df.append(entries, ignore_index=True)

df.dropna(axis=1, how='all', inplace=True)

In [8]:
df

,version_name,augment_1,augment_2,augment_3,placement,top 4,level,last_round,time_eliminated,trait_1,...,unit_2,unit_3,unit_4,unit_5,unit_6,unit_7,unit_8,unit_9,unit_10,unit_11
0,13.5,Illuminating Singularity,Ancient Archives I,Second Wind II,2,True,9,38,2243,Spellslinger 2,...,Annie 2,Rell 2,Yuumi 1,Nilah 2,Ekko 2,Taliyah 2,Janna 1,Syndra 2,NaN,NaN
1,13.5,A.D.M.I.N. Crest,Ancient Archives II,Chronobreak,1,True,9,38,2244,A.D.M.I.N. 3,...,Blitzcrank 3,Camille 2,Nilah 2,Alistar 1,Leblanc 2,Ekko 2,Taliyah 2,Janna 2,NaN,NaN
2,13.5,Portable Forge,Impact Velocity,Thrill of the Hunt II,1,True,8,37,2150,Ace 1,...,Jax 2,MissFortune 1,Ekko 2,AurelionSol 3,Sett 2,Sejuani 2,Sejuani 1,NaN,NaN,NaN
3,13.5,Three's Company,Relentless Assault,Pandora's Items,3,True,8,35,2058,A.D.M.I.N. 1,...,Vi 2,LeeSin 2,Riven 3,Jax 3,Leblanc 1,Zed 1,Sejuani 2,NaN,NaN,NaN
4,13.4,Consistency,Pandora's Bench,Upgrade: Berserk,1,True,8,37,2151,A.D.M.I.N. 2,...,Leblanc 2,Ekko 2,Ekko 2,Sejuani 2,Soraka 3,Leona 2,Fiddlesticks 1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,12.23,Three's Company,Vitality of the Ox,Makeshift Armor I,4,True,8,33,2031,Aegis 1,...,Yuumi 2,Nilah 2,Ekko 2,Taliyah 2,Aphelios 2,Syndra 1,Leona 2,NaN,NaN,NaN
246,12.23,Pandora's Items,Evasion,Battlemage II,1,True,8,41,2432,Ace 2,...,Jax 2,MissFortune 3,Ekko 1,Samira 2,Urgot 1,Leona 2,Mordekaiser 1,NaN,NaN,NaN
247,12.23,Renegade Heart,First Aid Kit I,Smash!,2,True,9,37,2120,Ace 1,...,Camille 2,Alistar 2,Ekko 1,Viego 2,Janna 2,Aphelios 1,Leona 2,Mordekaiser 1,NaN,NaN
248,12.23,Thrill of the Hunt II,Partners in Crime,Ox Force Crown,1,True,9,35,2111,Aegis 1,...,Annie 2,Alistar 2,Viego 2,Nunu 1,Urgot 2,Aphelios 2,Syndra 2,Leona 1,NaN,NaN


In [9]:
df.to_csv('cleaned_tft_data.csv')